# **SkinCancerClasification - No OverSampler With Scaling**

## **Import libraries, read dataset**

In [1]:
# Import our dependencies
import matplotlib.pyplot as plt
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('https://mlee22ph.github.io/Project4_Group11_AR_GP_ML/Resources/hmnist_28_28_RGB.csv')
data.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,192,153,193,195,155,192,197,154,185,202,...,173,124,138,183,147,166,185,154,177,2
1,25,14,30,68,48,75,123,93,126,158,...,60,39,55,25,14,28,25,14,27,2
2,192,138,153,200,145,163,201,142,160,206,...,167,129,143,159,124,142,136,104,117,2
3,38,19,30,95,59,72,143,103,119,171,...,44,26,36,25,12,17,25,12,15,2
4,158,113,139,194,144,174,215,162,191,225,...,209,166,185,172,135,149,109,78,92,2


In [3]:
y = data['label']
x = data.drop(columns = ['label'])

In [4]:
tabular_data = pd.read_csv('https://mlee22ph.github.io/Project4_Group11_AR_GP_ML/Resources/HAM10000_metadata.csv')
tabular_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [5]:
tabular_data.columns

Index(['lesion_id', 'image_id', 'dx', 'dx_type', 'age', 'sex', 'localization'], dtype='object')

In [6]:
classes = {4: ('nv', ' melanocytic nevi'), 6: ('mel', 'melanoma'), 2 :('bkl', 'benign keratosis-like lesions'),
           1:('bcc' , ' basal cell carcinoma'), 5: ('vasc', ' pyogenic granulomas and hemorrhage'), 0: ('akiec', 'Actinic keratoses and intraepithelial carcinomae'),
           3: ('df', 'dermatofibroma')}


## **Model Analysis and Prediction (Without Scaling and One Hot Encoding)**

In [7]:
x.shape

(10015, 2352)

In [8]:
y.shape

(10015,)

### **Standardization and Splitting Data**

In [9]:
# This process is often referred to as z-score normalization or standardization.
x = (x-np.mean(x))/np.std(x)
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size=0.2, random_state=1)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [10]:
print("Training Shape:", X_train.shape)
print("Testing Shape:", X_test.shape)

Training Shape: (8012, 2352)
Testing Shape: (2003, 2352)


### **Scaling and Normalization**

In [11]:
# Next, we normalize our training data to be between 0 and 1
scaler = MinMaxScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### **Create Sequential Model with 64 units**

In [12]:
num_classes = 64

# Create an empty sequential model
model = Sequential()
# Add the first layer where the input dimensions are the 784 pixel values
# We can also choose our activation function. `relu` is a common
model.add(Dense(num_classes, activation='relu', input_dim=X_train.shape[1]))
# Add a second hidden layer
model.add(Dense(num_classes, activation='relu'))
# Add our final output layer where the number of nodes
# corresponds to the number of y labels
model.add(Dense(7, activation='softmax'))

In [13]:
# We can summarize our model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                150592    
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 7)                 455       
                                                                 
Total params: 155207 (606.28 KB)
Trainable params: 155207 (606.28 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### **Compile Model using sparse_categorical_crossentropy, meaning no need for One Hot Encoding**

In [14]:
# Sparse categorical cross-entropy is an extension of the categorical cross-entropy loss function that is used when the output labels are represented in a sparse matrix format.
# In a sparse matrix format, the labels are represented as a single index value rather than a one-hot encoded vector. This means that the labels are integers rather than vectors.
# Both categorical cross-entropy and sparse categorical cross-entropy have the same loss function as defined above. The only difference between the two is in how labels are defined.
# Hint: your output layer in this example is using software for logistic regression (categorical)
# If your output layer activation was `linear` then you may want to use `mse` for loss
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit (train) the model
history = model.fit(
    X_train,
    Y_train,
    epochs=20,
    shuffle=True,
    verbose=2
)

Epoch 1/20
251/251 - 4s - loss: 1.0573 - accuracy: 0.6680 - 4s/epoch - 14ms/step
Epoch 2/20
251/251 - 2s - loss: 0.9746 - accuracy: 0.6714 - 2s/epoch - 6ms/step
Epoch 3/20
251/251 - 2s - loss: 0.9381 - accuracy: 0.6755 - 2s/epoch - 6ms/step
Epoch 4/20
251/251 - 2s - loss: 0.9172 - accuracy: 0.6784 - 2s/epoch - 6ms/step
Epoch 5/20
251/251 - 3s - loss: 0.9093 - accuracy: 0.6805 - 3s/epoch - 10ms/step
Epoch 6/20
251/251 - 2s - loss: 0.9044 - accuracy: 0.6807 - 2s/epoch - 9ms/step
Epoch 7/20
251/251 - 1s - loss: 0.8840 - accuracy: 0.6862 - 885ms/epoch - 4ms/step
Epoch 8/20
251/251 - 1s - loss: 0.8786 - accuracy: 0.6868 - 820ms/epoch - 3ms/step
Epoch 9/20
251/251 - 1s - loss: 0.8690 - accuracy: 0.6911 - 808ms/epoch - 3ms/step
Epoch 10/20
251/251 - 1s - loss: 0.8698 - accuracy: 0.6888 - 792ms/epoch - 3ms/step
Epoch 11/20
251/251 - 1s - loss: 0.8676 - accuracy: 0.6905 - 799ms/epoch - 3ms/step
Epoch 12/20
251/251 - 1s - loss: 0.8464 - accuracy: 0.6950 - 792ms/epoch - 3ms/step
Epoch 13/20
251/2

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test, Y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

## **Making Predictions and Classification Report**

In [ ]:
plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
y_true = np.array(Y_test)
y_pred = model.predict(X_test)
y_pred = np.array(list(map(lambda x: np.argmax(x), y_pred)))
# Print result and prediction for side by side comparison
print(y_true)
print(y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report

target_names = ["akiec", "bcc", "bkl", "df", "nv", "vasc", "mel"]
report = classification_report(y_true, y_pred, target_names=target_names)

print("\nClassification Report:")
print(report)

In [ ]:
# Print confusion matrix
confusion_matrix(y_true, y_pred)

## **Saving and Loading models**

In [ ]:
# Save the model
model.save("SkinCancerClasificationNoOverSamplerWithScaling.h5")

In [ ]:
# Load the model
from tensorflow.keras.models import load_model
model = load_model("SkinCancerClasificationNoOverSamplerWithScaling.h5")